In [ ]:
import os
import sys
os.environ['GLEAMS_HOME'] = os.path.join(os.environ['HOME'],
                                         'Projects/gleams')
# Make sure all code is in the PATH.
sys.path.append(
    os.path.normpath(os.path.join(os.environ['GLEAMS_HOME'], 'src')))

In [ ]:
import warnings
from sklearn.exceptions import EfficiencyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=EfficiencyWarning)

In [ ]:
import collections
import copy
import itertools
import shutil
import time

import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import psutil
import pyteomics
import seaborn as sns
import tqdm.notebook as tqdm
from sklearn.metrics import homogeneity_score, completeness_score

In [ ]:
# Initialize logging.
from gleams import logger as glogger
glogger.init()
# Initialize all random seeds before importing any packages.
from gleams import rndm
rndm.set_seeds()

from gleams import config
from gleams.cluster import cluster
from gleams.feature import feature, spectrum
from gleams.ms_io import ms_io
from gleams.nn import nn

In [ ]:
import logging
logger = logging.getLogger('gleams')
logger.setLevel(logging.DEBUG)

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

## Export spectra

In [ ]:
! mkdir -p $GLEAMS_HOME/notebooks/cluster_comparison

In [ ]:
cluster_dir = os.path.join(
    os.environ['GLEAMS_HOME'], 'notebooks', 'cluster_comparison')

In [ ]:
def get_spectra_from_file(dataset, filename, scans):
    logger.debug('Process file %s/%s', dataset, filename)
    peak_filename = os.path.join(os.environ['GLEAMS_HOME'], 'data', 'peak',
                                 dataset, filename)
    if os.path.isfile(peak_filename):
        return [spec for spec in ms_io.get_spectra(peak_filename, scans)
                if spectrum.preprocess(copy.deepcopy(spec),
                                       config.fragment_mz_min,
                                       config.fragment_mz_max).is_valid]
    else:
        return None

In [ ]:
split = 'test'
max_spectra_per_split = 20_000_000

In [ ]:
datasets = pd.read_parquet(
    os.path.join(os.environ['GLEAMS_HOME'], 'data', 'embed',
                 f'embed_{config.massivekb_task_id}_{split}.parquet'))
datasets_splits = np.array_split(
    datasets.sample(frac=1),
    max(3, len(datasets) // max_spectra_per_split + 1))

psms = pd.read_parquet(
    os.path.join(os.environ['GLEAMS_HOME'], 'data', 'metadata',
                 f'massivekb_ids_{config.massivekb_task_id}.parquet'),
    columns=['dataset', 'filename', 'scan', 'sequence'])
    
for split_i, datasets_split in enumerate(datasets_splits):
    cluster_dir_split = os.path.join(cluster_dir, str(split_i))
    if not os.path.exists(cluster_dir_split):
        os.makedirs(cluster_dir_split)
    filename_mgf = os.path.join(cluster_dir_split, 'cluster_comparison.mgf')
    filename_metadata = os.path.join(cluster_dir_split,
                                     'cluster_comparison.parquet')
    if not os.path.isfile(filename_metadata):
        logger.info('Partition %d/%d: Export spectra to be clustered to MGF '
                    'file', split_i + 1, len(datasets_splits))
        if os.path.isfile(filename_mgf):
            os.remove(filename_mgf)
        spectrum_idx = []
        dataset_filename_scans = (datasets_split
                                  .groupby(['dataset', 'filename'])['scan']
                                  .apply(list).reset_index())
        for dataset, filename, spectra in tqdm.tqdm(
                zip(dataset_filename_scans['dataset'],
                    dataset_filename_scans['filename'],
                    joblib.Parallel(n_jobs=-1, backend='multiprocessing')(
                        joblib.delayed(get_spectra_from_file)
                        (dataset, filename, scans)
                        for dataset, filename, scans in zip(
                            dataset_filename_scans['dataset'],
                            dataset_filename_scans['filename'],
                            dataset_filename_scans['scan']))),
                desc='Files processed', total=len(dataset_filename_scans)):
            if spectra is not None:
                spectra_dicts = []
                for spec in spectra:
                    if (config.charges[0] <= spec.precursor_charge
                            <= config.charges[1]):
                        spectra_dicts.append(
                            {'m/z array': spec.mz,
                            'intensity array': spec.intensity,
                            'params': {
                                'TITLE': len(spectrum_idx),
                                'RTINSECONDS': spec.retention_time,
                                'PEPMASS': spec.precursor_mz,
                                'CHARGE': f'{spec.precursor_charge}+'}})
                        spectrum_idx.append(
                            (dataset, filename, int(spec.identifier),
                             spec.precursor_charge, spec.precursor_mz))
                with open(filename_mgf, 'a') as f:
                    pyteomics.mgf.write(spectra_dicts, f)
        metadata = pd.merge(
            pd.DataFrame(spectrum_idx, columns=['dataset', 'filename', 'scan',
                                                'charge', 'mz']),
            psms, 'left', ['dataset', 'filename', 'scan'])
        metadata['sequence'] = (metadata['sequence'].str.replace('I', 'L')
                                + '/' + metadata['charge'].astype(str))
        metadata.to_parquet(filename_metadata)

## Cluster

In [ ]:
min_cluster_sizes = [(2, None), (5, None), (10, None), (50, None)]
min_peptide_size = None

In [ ]:
def _count_majority_label_mismatch(labels):
    labels_assigned = labels.dropna()
    if len(labels_assigned) <= 1:
        return 0
    else:
        return len(labels_assigned) - labels_assigned.value_counts().iat[0]


def evaluate_clusters(clusters, min_cluster_size=None, max_cluster_size=None):
    clusters = clusters.copy()
    # Only consider clusters with specific minimum (inclusive) and/or
    # maximum (exclusive) size.
    cluster_counts = clusters['cluster'].value_counts(dropna=False)
    if min_cluster_size is not None:
        clusters.loc[clusters['cluster'].isin(cluster_counts[
            cluster_counts < min_cluster_size].index), 'cluster'] = -1
    if max_cluster_size is not None:
        clusters.loc[clusters['cluster'].isin(cluster_counts[
            cluster_counts >= max_cluster_size].index), 'cluster'] = -1

    # Use consecutive cluster labels, skipping the noise points.    
    cluster_map = (clusters['cluster'].value_counts(dropna=False)
                   .drop(index=-1).to_frame().reset_index().reset_index()
                   .rename(columns={'index': 'old', 'level_0': 'new'})
                   .set_index('old')['new'])
    cluster_map = cluster_map.to_dict(collections.defaultdict(lambda: -1))
    clusters['cluster'] = clusters['cluster'].map(cluster_map)
    num_clusters = clusters['cluster'].max() + 1

    # Reassign noise points to singleton clusters.
    noise_mask = clusters['cluster'] == -1
    num_noise = noise_mask.sum()
    clusters.loc[noise_mask, 'cluster'] = np.arange(
        num_clusters, num_clusters + num_noise)

    # Compute cluster evaluation measures.
    prop_clustered = (len(clusters) - num_noise) / len(clusters)

    clusters_ident = clusters.dropna(subset=['sequence'])
    clusters_ident_non_noise = (clusters[~noise_mask]
                                .dropna(subset=['sequence']))

    # The number of incorrectly clustered spectra is the number of PSMs that
    # differ from the majority PSM. Unidentified spectra are not considered.
    prop_clustered_incorrect = sum(joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_count_majority_label_mismatch)(clust['sequence'])
        for _, clust in clusters[~noise_mask].groupby('cluster')))
    prop_clustered_incorrect /= len(clusters_ident_non_noise)

    # Homogeneity measures whether clusters contain only identical PSMs.
    # This is only evaluated on non-noise points, because the noise cluster
    # is highly non-homogeneous by definition.
    homogeneity = homogeneity_score(clusters_ident_non_noise['sequence'],
                                    clusters_ident_non_noise['cluster'])
    # Completeness measures whether identical PSMs are assigned to the same
    # cluster.
    # This is evaluated on all PSMs, including those clustered as noise.
    completeness = completeness_score(clusters_ident['sequence'],
                                      clusters_ident['cluster'])

    return (len(clusters) - num_noise, num_noise,
            prop_clustered, prop_clustered_incorrect,
            homogeneity, completeness)

In [ ]:
def _get_clusters_mscluster(dir_clusters, sequences):
    cluster_labels, cluster_i = np.full(len(sequences), -1), -1
    for filename in os.listdir(dir_clusters):
        if filename.endswith('.clust'):
            with open(os.path.join(dir_clusters, filename)) as f_in:
                for line in f_in:
                    if line.startswith('mscluster'):
                        cluster_i += 1
                    elif not line.isspace():
                        cluster_labels[int(line.split('\t')[2])] = cluster_i
    return pd.DataFrame({'sequence': sequences, 'cluster': cluster_labels})


def _get_clusters_spectracluster(filename_clusters, sequences):
    cluster_labels, cluster_i = np.full(len(sequences), -1), -1
    with open(filename_clusters) as f_in:
        for line in f_in:
            if line.startswith('=Cluster='):
                cluster_i += 1
            elif line.startswith('SPEC'):
                cluster_labels[
                    int(line[line.find('#id=index=') + len('#id=index='):
                             line.find('#title')]) - 1] = cluster_i
    return pd.DataFrame({'sequence': sequences, 'cluster': cluster_labels})


def _get_clusters_gleams(filename_clusters, sequences):
    return pd.DataFrame({'sequence': sequences,
                         'cluster': np.load(filename_clusters)})

In [ ]:
performance = []

In [ ]:
hyper_param = {
    'MS-Cluster': list(itertools.product(
        [0.0001, 0.001, 0.01, 0.1], [3, 10])),
    'spectra-cluster': list(itertools.product(
        [0.9999, 0.999, 0.99, 0.95, 0.9], [3, 10])),
    'GLEAMS': list(itertools.product(np.arange(0.02, 0.08, 0.005), [2]))}

### MS-Cluster

MS-Cluster hyperparameters that influence the clustering quality are:

- `--mixture-prob <X>`: the probability wrongfully adding a spectrum to a cluster (default X=0.05)
- `--num-rounds <X>`: determines how many rounds are used for the hierarchical clustering (default X=3).

In [ ]:
for split_i in os.listdir(cluster_dir):
    metadata = pd.read_parquet(os.path.join(cluster_dir, split_i,
                                            'cluster_comparison.parquet'))
    dir_mscluster = os.path.join(cluster_dir, split_i, 'mscluster')
    # MS-Cluster preprocessing.
    if not os.path.exists(dir_mscluster):
        os.makedirs(dir_mscluster)
    cmd = f"""ls {os.path.join(cluster_dir, split_i)}/cluster_comparison.mgf \
        > {dir_mscluster}/mscluster_spec_list.txt"""
    ! eval {cmd}
    cmd = f"""$GLEAMS_HOME/bin/MsCluster/MsCluster \
        --model LTQ_TRYP \
        --list {dir_mscluster}/mscluster_spec_list.txt \
        --output-name mscluster \
        --tmp-dir {dir_mscluster}/dat \
        --out-dir {dir_mscluster} \
        --dat-only \
        --model-dir $GLEAMS_HOME/bin/MsCluster/Models \
        --keep-dat \
        --assign-charges"""
    ! eval {cmd}
    # MS-Cluster clustering.
    for i, (mixture_prob, num_rounds) in enumerate(hyper_param['MS-Cluster']):
        logger.info('MS-Cluster run %d (mixture-prob=%.4f ; num-rounds=%d)',
                    i + 1, mixture_prob, num_rounds)
        # Execute clustering.
        cmd = f"""$GLEAMS_HOME/bin/MsCluster/MsCluster \
            --model LTQ_TRYP \
            --dat-list {dir_mscluster}/dat/mscluster_dat_list.txt \
            --output-name mscluster \
            --output-file-size {max_spectra_per_split} \
            --out-dir {dir_mscluster}/cluster_{i} \
            --model-dir $GLEAMS_HOME/bin/MsCluster/Models \
            --memory-gb {int(psutil.virtual_memory().available / (1024 ** 3))} \
            --fragment-tolerance 0.05 \
            --precursor-ppm 10 \
            --assign-charges \
            --mixture-prob {mixture_prob} \
            --num-rounds {num_rounds} \
            --keep-dataset-idx"""
        if not os.path.isfile(os.path.join(dir_mscluster, f'cluster_{i}',
                                           'mscluster_0_0_mgf_list.txt')):
            ! eval {cmd}
        # Account for failed MS-Cluster runs.
        if not os.path.isfile(os.path.join(dir_mscluster, f'cluster_{i}',
                                           'mscluster_0_0_mgf_list.txt')):
            continue
        # Evaluate clustering performance.
        clusters = _get_clusters_mscluster(
            os.path.join(dir_mscluster, f'cluster_{i}', 'clust'),
            metadata['sequence'])
        for min_cluster_size, max_cluster_size in min_cluster_sizes:
            num_clustered, num_noise, \
                prop_clustered, prop_clustered_incorrect, \
                homogeneity, completeness = \
                    evaluate_clusters(clusters, min_cluster_size,
                                      max_cluster_size)
            performance.append(('MS-Cluster', (mixture_prob, num_rounds),
                                min_cluster_size, max_cluster_size,
                                num_clustered, num_noise,
                                prop_clustered, prop_clustered_incorrect,
                                homogeneity, completeness))

### spectra-cluster

spectra-cluster hyperparameters that influence the clustering quality are:

- `-rounds <arg>`: number of clustering rounds to use.
- `-threshold_end <arg>`: (lowest) final clustering threshold
- `-threshold_start <arg>`: (highest) starting threshold

In [ ]:
for split_i in os.listdir(cluster_dir):
    metadata = pd.read_parquet(os.path.join(cluster_dir, split_i,
                                            'cluster_comparison.parquet'))
    dir_spectracluster = os.path.join(cluster_dir, split_i, 'spectra-cluster')
    if not os.path.exists(os.path.join(dir_spectracluster, 'tmp')):
        os.makedirs(os.path.join(dir_spectracluster, 'tmp'))
    # spectra-cluster clustering.
    for i, (threshold_end, rounds) in enumerate(hyper_param['spectra-cluster']):
        logger.info('spectra-cluster run %d (threshold_end=%.4f ; rounds=%d)',
                    i + 1, threshold_end, rounds)
        # Execute clustering.
        cmd = f"""java -jar $GLEAMS_HOME/bin/spectra-cluster/spectra-cluster-cli-1.1.2.jar \
            {os.path.join(cluster_dir, split_i)}/cluster_comparison.mgf \
            -binary_directory {dir_spectracluster}/tmp \
            -fast_mode \
            -fragment_tolerance 0.05 \
            -keep_binary_files \
            -major_peak_jobs $(nproc --all) \
            -output_path {dir_spectracluster}/clusters_{i}.txt \
            -precursor_tolerance 10 \
            -precursor_tolerance_unit ppm \
            -reuse_binary_files \
            -rounds {rounds} \
            -threshold_end {threshold_end} \
            -threshold_start 1.0 \
            -x_disable_mgf_comments"""
        if not os.path.isfile(os.path.join(dir_spectracluster,
                                           f'clusters_{i}.txt')):
            ! eval {cmd}
        # Evaluate clustering performance.
        clusters = _get_clusters_spectracluster(
            os.path.join(dir_spectracluster, f'clusters_{i}.txt'),
            metadata['sequence'])
        for min_cluster_size, max_cluster_size in min_cluster_sizes:
            num_clustered, num_noise, \
                prop_clustered, prop_clustered_incorrect, \
                homogeneity, completeness = \
                    evaluate_clusters(clusters, min_cluster_size,
                                      max_cluster_size)
            performance.append(('spectra-cluster', (threshold_end, rounds),
                                min_cluster_size, max_cluster_size,
                                num_clustered, num_noise,
                                prop_clustered, prop_clustered_incorrect,
                                homogeneity, completeness))

### GLEAMS

GLEAMS hyperparameters that influence the clustering quality are ([Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)):

- `eps`: The maximum distance between two samples for one to be considered as in the neighborhood of the other. This is not a maximum bound on the distances of points within a cluster. This is the most important DBSCAN parameter to choose appropriately for your data set and distance function.
- `min_samples`:     The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

In [ ]:
for split_i in os.listdir(cluster_dir):
    metadata_filename = os.path.join(cluster_dir, split_i,
                                     'cluster_comparison.parquet')
    metadata = pd.read_parquet(metadata_filename)
    dir_gleams = os.path.join(cluster_dir, split_i, 'gleams')
    embed_filename = os.path.join(dir_gleams, 'embed_cluster_comparison.npy')
    dist_filename = os.path.join(dir_gleams, 'dist_cluster_comparison.npz')
    clust_filename = os.path.join(dir_gleams, 'clusters_cluster_comparison.npy')
    # GLEAMS preprocessing.
    if not os.path.exists(dir_gleams):
        os.makedirs(dir_gleams)
    # Extract the relevant entries from all (previously computed) embeddings.
    if not os.path.isfile(embed_filename):
        embed_idx = pd.merge(
            metadata, pd.read_parquet(
                os.path.join(
                    os.environ['GLEAMS_HOME'], 'data', 'embed',
                    f'embed_{config.massivekb_task_id}_{split}.parquet'),
                columns=['dataset', 'filename', 'scan']).reset_index(),
            'left', ['dataset', 'filename', 'scan'])['index'].values
        embeddings = np.load(os.path.join(
            os.environ['GLEAMS_HOME'], 'data', 'embed',
            f'embed_{config.massivekb_task_id}_{split}.npy'), mmap_mode='r')
        np.save(embed_filename, embeddings[embed_idx])
    # Compute pairwise distances.
    if not os.path.isfile(dist_filename):
        cluster.compute_pairwise_distances(
            embed_filename, metadata_filename, config.charges)
        os.rename(os.path.join(os.environ['GLEAMS_HOME'], 'data', 'cluster',
                               'dist_cluster_comparison.npz'),
                  dist_filename)
        os.rename(os.path.join(os.environ['GLEAMS_HOME'], 'data', 'cluster',
                               os.path.basename(metadata_filename)),
                  metadata_filename)
        os.rename(os.path.join(os.environ['GLEAMS_HOME'], 'data', 'cluster',
                               os.path.basename(embed_filename)),
                  embed_filename)
        metadata = pd.read_parquet(metadata_filename)
    # GLEAMS clustering.
    for i, (eps, min_samples) in enumerate(hyper_param['GLEAMS']):
        logger.info('GLEAMS run %d (eps=%.3f ; min_samples=%d)',
                    i + 1, eps, min_samples)
        if os.path.isfile(clust_filename):
            os.remove(clust_filename)
        config.eps, config.min_samples = eps, min_samples
        cluster_filename_i = clust_filename.replace('.npy', f'_{i}.npy')
        # Execute clustering.
        if not os.path.isfile(cluster_filename_i):
            cluster.cluster(dist_filename, metadata_filename)
            # Rename file to retain clustering results.
            os.rename(clust_filename, cluster_filename_i)
        # Evaluate clustering performance.
        clusters = _get_clusters_gleams(cluster_filename_i,
                                        metadata['sequence'])
        for min_cluster_size, max_cluster_size in min_cluster_sizes:
            num_clustered, num_noise, \
                prop_clustered, prop_clustered_incorrect, \
                homogeneity, completeness = \
                    evaluate_clusters(clusters, min_cluster_size,
                                      max_cluster_size)
            performance.append(('GLEAMS', (eps, min_samples),
                                min_cluster_size, max_cluster_size,
                                num_clustered, num_noise,
                                prop_clustered, prop_clustered_incorrect,
                                homogeneity, completeness))
    # Clean up ANN indexes.
    ann_dir = os.path.join(os.environ['GLEAMS_HOME'], 'data', 'cluster', 'ann')
    for filename in os.listdir(ann_dir):
        if filename.startswith('ann_cluster_comparison_'):
            os.remove(os.path.join(ann_dir, filename))

## Compare clustering results

In [ ]:
performance = pd.DataFrame(performance, columns=[
    'tool', 'hyperparameters',
    'min_cluster_size', 'max_cluster_size',
    'num_clustered', 'num_noise',
    'prop_clustered', 'prop_clustered_incorrect',
    'homogeneity', 'completeness'])
performance.to_csv('cluster_comparison.csv', index=False)

In [ ]:
# performance = pd.read_csv('cluster_comparison.csv')

In [ ]:
def get_pareto_frontier_idx(arr, threshold0=0, threshold1=0):
    # Sort by the first column.
    order = arr[:, 0].argsort()
    arr_sorted = arr[order]
    # Iteratively add points to the Pareto frontier.
    pareto_idx = [0]
    for i in range(1, arr_sorted.shape[0]):
        if (arr_sorted[i, 0] > (arr_sorted[pareto_idx[-1], 0]
                                + threshold0) and
                arr_sorted[i, 1] > (arr_sorted[pareto_idx[-1], 1]
                                    + threshold1)):
            pareto_idx.append(i)
    return order[pareto_idx]

In [ ]:
width = 7
height = width / 1.618
fig, axes = plt.subplots(1, 2, figsize=(width * 2, height))
axes = np.ravel(axes)

# Number of clustered spectra and completeness.
cluster_size = 2
for tool in ('GLEAMS', 'MS-Cluster', 'spectra-cluster'):
    tool_performance = performance[
        (performance['tool'] == tool) &
        (performance['min_cluster_size'] == cluster_size)]
    axes[0].errorbar(
        x=(tool_performance.groupby('hyperparameters')
           ['prop_clustered_incorrect'].mean()),
        y=(tool_performance.groupby('hyperparameters')
           ['prop_clustered'].mean()),
        xerr=(tool_performance.groupby('hyperparameters')
              ['prop_clustered_incorrect'].std()),
        yerr=(tool_performance.groupby('hyperparameters')
              ['prop_clustered'].std()),
        marker='o', label=tool)
    axes[1].errorbar(
        x=(tool_performance.groupby('hyperparameters')
           ['prop_clustered_incorrect'].mean()),
        y=(tool_performance.groupby('hyperparameters')
           ['completeness'].mean()),
        xerr=(tool_performance.groupby('hyperparameters')
              ['prop_clustered_incorrect'].std()),
        yerr=(tool_performance.groupby('hyperparameters')
              ['completeness'].std()),
        marker='o', label=tool)

axes[0].set_xlim(0, 0.05)
axes[0].set_ylim(0, 1)
axes[1].set_xlim(0, 0.05)
axes[1].set_ylim(0.85, 1)

axes[0].xaxis.set_major_formatter(mticker.PercentFormatter(1, 0))
axes[0].yaxis.set_major_formatter(mticker.PercentFormatter(1, 0))
axes[1].xaxis.set_major_formatter(mticker.PercentFormatter(1, 0))

axes[0].set_xlabel('Incorrectly clustered spectra')
axes[0].set_ylabel('Clustered spectra')
axes[1].set_xlabel('Incorrectly clustered spectra')
axes[1].set_ylabel('Completeness')

axes[0].legend(loc='lower right', frameon=False)
axes[1].legend(loc='lower right', frameon=False)
    
fig.tight_layout()

sns.despine()

plt.savefig('cluster_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
logging.shutdown()